# 正則化

*Regularization*



In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Ridge回帰

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
scaler = StandardScaler()
X, y = load_diabetes(return_X_y=True)
X = scaler.fit_transform(X)

In [3]:
class RidgeRegression:
    def __init__(self, alpha=1.):
        self.alpha = alpha
        self.weights = None

    def fit(self, X, y):
        X = np.insert(X, 0, 1, axis=1)
        I = np.identity(X.shape[1], dtype=np.float32)
        I[0, 0] = 0 # バイアス項は無視
        self.weights = np.linalg.inv(X.T @ X + self.alpha*I) @ X.T @ y

    def predict(self, X):
        X = np.insert(X, 0, 1, axis=1)
        return X @ self.weights

In [4]:
model = RidgeRegression(10)
model.fit(X, y)
y_pred = model.predict(X)
print(mean_squared_error(y, y_pred))
model.weights

2872.2027699648543


array([152.13348416,  -0.257949  , -10.93635667,  24.60009446,
        15.09438258, -11.29561827,   1.80876776,  -6.56180515,
         5.6004003 ,  25.33209609,   3.52291212])

In [5]:
from sklearn.linear_model import Ridge
correct_model = Ridge(alpha=10.)
correct_model.fit(X, y)
y_pred = correct_model.predict(X)
print(mean_squared_error(y, y_pred))
correct_model.coef_

2872.2027699648543


array([ -0.257949  , -10.93635667,  24.60009446,  15.09438258,
       -11.29561827,   1.80876776,  -6.56180515,   5.6004003 ,
        25.33209609,   3.52291212])

In [6]:
correct_model.intercept_

152.13348416289594

## Lasso回帰

In [7]:
# x = np.linspace(-2, 5, 100)
# y = (5 - x) ** 2 + 5 * np.abs(x)
# plt.plot(x, y)

In [16]:
class LassoRegression:
    def __init__(self, alpha: float = 1., max_iter: int = 1000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.weights = None

    @staticmethod
    def soft_threshold(x, thre):
        if x > thre:
            return x - thre
        elif x < -thre:
            return x + thre
        else:
            return 0

    def fit(self, X, y):
        # X = np.insert(X, 0, 1, axis=1)
        n_features = X.shape[1]
        self.weights = np.zeros(n_features)
        for _ in range(self.max_iter):
            for i in range(n_features):
                weights = self.weights.copy()
                weights[i] = 0
                r = y - X @ weights
                c = X[:, i] @ r
                a = np.linalg.norm(X[:, i]) ** 2
                self.weights[i] = self.soft_threshold(c / a, self.alpha)

    def predict(self, X):
        # X = np.insert(X, 0, 1, axis=1)
        return X @ self.weights

In [18]:
model = LassoRegression(1.)
model.fit(X, y)
y_pred = model.predict(X)
print(mean_squared_error(y, y_pred))
model.weights

26030.765777093777


array([  0.        ,  -9.31932954,  24.83150373,  14.08898551,
        -4.83894619,   0.        , -10.6227563 ,   0.        ,
        24.4209334 ,   2.56187551])

In [10]:
from sklearn.linear_model import Lasso

In [19]:
correct_model = Lasso(alpha=1., fit_intercept=False)
correct_model.fit(X, y)
y_pred = correct_model.predict(X)
print(mean_squared_error(y, y_pred))
correct_model.coef_

26030.76631319103


array([ -0.        ,  -9.31941253,  24.83127631,  14.08870568,
        -4.83892808,  -0.        , -10.62279919,   0.        ,
        24.42081057,   2.56212987])